# Lab

In [1]:
%load_ext watermark
%watermark -v -p numpy,pandas,polars,omegaconf --conda

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

numpy    : 1.26.4
pandas   : 2.2.1
polars   : 0.20.18
omegaconf: 2.3.0

conda environment: torch_p11



In [2]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "info": "#76FF7B",
        "warning": "#FBDDFE",
        "error": "#FF0000",
    }
)
console = Console(theme=custom_theme)

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")


# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F

In [4]:
seed: int = 123

GPT_CONFIG_124M: dict[str, Any] = {
    "vocab_size": 50_257,
    "context_length": 1_024,
    "emb_dim": 768,
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,
    "drop_rate": 0.1,  # Dropout rate
    "qkv_bias": False,
}

In [5]:
class SelfAttention(nn.Module):
    def __init__(self, in_feats: int, out_feats: int, qkv_bias: bool = False) -> None:
        super().__init__()

        # Size: (seq_len, emb_dim)
        self.query_weights = nn.Linear(in_feats, out_feats, bias=qkv_bias)
        self.key_weights = nn.Linear(in_feats, out_feats, bias=qkv_bias)
        self.value_weights = nn.Linear(in_feats, out_feats, bias=qkv_bias)

    def forward(self, x: Tensor) -> Tensor:
        # b_size, seq_len, emb_dim = x.shape
        # (b_size, emb_dim, seq_len) @ (seq_len, emb_dim) -> (b_size, emb_dim, emb_dim)
        query = self.query_weights(x)
        key = self.key_weights(x)
        value = self.value_weights(x)

        # Attention scores
        # (b_size, emb_dim, seq_len) @ (seq_len, emb_dim) -> (b_size, emb_dim, emb_dim)
        attn_scores: Tensor = torch.matmul(query, key.transpose(-1, -2))
        attn_weights: Tensor = F.softmax(attn_scores / key.shape[1] ** 0.5, dim=-1)
        # (seq_len, emb_dim) @ (b_size, emb_dim, emb_dim) -> (b_size, seq_len, emb_dim)
        context_vector: Tensor = torch.matmul(attn_weights, value)
        return context_vector

In [6]:
vocab_size: int = 27
embedding_dim: int = 16
context_size: int = 8
batch_size: int = 2

input_seq: Tensor = torch.rand(
    size=(batch_size, context_size, embedding_dim), dtype=torch.float32
)
self_attn: SelfAttention = SelfAttention(embedding_dim, embedding_dim)
context_vector: Tensor = self_attn(input_seq)
context_vector

tensor([[[-0.0667,  0.0210,  0.2596,  0.0146, -0.1486, -0.5148,  0.1315,
           0.0073,  0.4838, -0.0732,  0.0407, -0.1094,  0.0654, -0.1763,
          -0.1097, -0.3647],
         [-0.0678,  0.0146,  0.2611,  0.0196, -0.1453, -0.5066,  0.1322,
           0.0152,  0.4790, -0.0674,  0.0298, -0.1018,  0.0607, -0.1728,
          -0.1094, -0.3596],
         [-0.0721,  0.0176,  0.2638,  0.0138, -0.1494, -0.5155,  0.1304,
           0.0085,  0.4806, -0.0696,  0.0386, -0.1083,  0.0672, -0.1790,
          -0.1091, -0.3607],
         [-0.0645,  0.0211,  0.2616,  0.0160, -0.1461, -0.5140,  0.1289,
           0.0134,  0.4836, -0.0691,  0.0336, -0.1075,  0.0604, -0.1737,
          -0.1141, -0.3659],
         [-0.0608,  0.0238,  0.2599,  0.0174, -0.1477, -0.5108,  0.1299,
           0.0073,  0.4837, -0.0751,  0.0407, -0.1087,  0.0602, -0.1727,
          -0.1105, -0.3635],
         [-0.0603,  0.0217,  0.2532,  0.0200, -0.1478, -0.5086,  0.1314,
           0.0057,  0.4855, -0.0746,  0.0415, -0.109

In [7]:
class CausalSelfAttention(nn.Module):
    def __init__(
        self,
        d_model: int,
        context_size: int,
        dropout_pct: float = 0.0,
        qkv_bias: bool = False,
    ) -> None:
        super().__init__()

        # Size: (seq_len, emb_dim)
        self.query_weights = nn.Linear(d_model, d_model, bias=qkv_bias)
        self.key_weights = nn.Linear(d_model, d_model, bias=qkv_bias)
        self.value_weights = nn.Linear(d_model, d_model, bias=qkv_bias)

        self.register_buffer(
            "mask", torch.triu(torch.ones(context_size, context_size), diagonal=1)
        )
        self.dropout = nn.Dropout(p=dropout_pct)

    def forward(self, x: Tensor) -> Tensor:
        b_size, seq_len, emb_dim = x.shape
        # (b_size, emb_dim, seq_len) @ (seq_len, emb_dim) -> (b_size, emb_dim, emb_dim)
        query = self.query_weights(x)
        key = self.key_weights(x)
        value = self.value_weights(x)

        # Attention scores
        # (b_size, emb_dim, seq_len) @ (seq_len, emb_dim) -> (b_size, emb_dim, emb_dim)
        attn_scores: Tensor = torch.matmul(query, key.transpose(-1, -2))
        # Apply mask (inplace). The slicing ensures that the seq_len is consistent across the batch.
        attn_scores.masked_fill_(self.mask.bool()[:seq_len, :seq_len], -torch.inf)

        attn_weights: Tensor = F.softmax(attn_scores / key.shape[1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        # (seq_len, emb_dim) @ (b_size, emb_dim, emb_dim) -> (b_size, seq_len, emb_dim)
        context_vector: Tensor = torch.matmul(attn_weights, value)
        return context_vector

In [8]:
torch.manual_seed(seed)

input_seq: Tensor = torch.rand(
    size=(batch_size, context_size, embedding_dim), dtype=torch.float32
)
causal_self_attn: CausalSelfAttention = CausalSelfAttention(
    d_model=embedding_dim, context_size=context_size, dropout_pct=0.1
)
context_vector: Tensor = causal_self_attn(input_seq)
context_vector.shape

torch.Size([2, 8, 16])

In [9]:
class SequentialMultiHeadAttention(nn.Module):
    def __init__(
        self,
        d_model: int,
        context_size: int,
        num_heads: int,
        dropout: float = 0.0,
        qkv_bias: bool = False,
    ):
        super().__init__()

        self.heads = nn.ModuleList(
            [
                CausalSelfAttention(d_model, context_size, dropout, qkv_bias)
                for _ in range(num_heads)
            ]
        )

    def forward(self, x: Tensor) -> Tensor:
        # Concat along the feature (emb) dimension
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [10]:
torch.manual_seed(seed)

multi_head_attn: SequentialMultiHeadAttention = SequentialMultiHeadAttention(
    d_model=embedding_dim,
    context_size=context_size,
    num_heads=3,
    dropout=0.1,
)
print(f"{input_seq.shape = }")
print(f"{multi_head_attn = }")
output: Tensor = multi_head_attn(input_seq)
print(f"{output.shape = }")

input_seq.shape = torch.Size([2, 8, 16])
multi_head_attn = SequentialMultiHeadAttention(
  (heads): ModuleList(
    (0-2): 3 x CausalSelfAttention(
      (query_weights): Linear(in_features=16, out_features=16, bias=False)
      (key_weights): Linear(in_features=16, out_features=16, bias=False)
      (value_weights): Linear(in_features=16, out_features=16, bias=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)
output.shape = torch.Size([2, 8, 48])


<hr><br><br>

### Multi-head Attention

- Instead of relying on a `single attention mechanism`, `multi-head attention` uses multiple "`heads`" that work in parallel.
- Each `head` analyzes the input sequence from a `slightly different perspective`.
- These individual analyses are then `combined` (concatenated) to create a `richer understanding` of the relationships between elements in the sequence.

#### Here's a breakdown of the key points with clarification:

- **`Causal self-attention`**: This refers to a type of attention where an element in the sequence only attends to the elements that come before it in the sequence.

- **`Multiple heads in parallel`**: The core concept of `Multi-head Attention`. Instead of one attention mechanism, multiple "heads" analyze the data simultaneously.

- **`Input sequence split and processed`**: Each head gets a portion of the original input data (`d_model`) based on the number of heads (`num_heads`). This creates a lower dimension for each head (`head_dim`) for processing.

- **`Concatenation`**: After each head analyzes its portion of the data, the results are combined (concatenated) to create a richer representation that captures insights from all the heads.
  - E.g. 
    - With a `d_model` of 64 (original input has 64 features) and 4 heads, each head gets 16 dimensions (features) to process (64 / 4). 
    - These 4 heads analyze the data in `parallel`, and then their outputs are `combined` to create a `final representation` with potentially deeper understanding than a single head could achieve.

In [11]:
class MultiHeadAttention(nn.Module):
    """
    A Multi-Head Attention layer for use in neural network architectures.

    Args:
        d_model (int): The dimension of the input and output features.
        context_size (int): The size of the context window (neighborhood considered for attention).
        num_heads (int): The number of heads used in the Multi-Head Attention.
        dropout_pct (float, optional): The dropout probability for the attention weights. Defaults to 0.1.
        qkv_bias (bool, optional): Whether to add bias terms to the linear transformations for queries, keys,
        and values. Defaults to False.

    Raises:
        AssertionError: If `d_model` is not divisible by `num_heads`.

    Shapes:
        - Input: (batch_size, seq_len, d_model)
        - Output: (batch_size, seq_len, d_model)

    Note:
        B, T, C: (batch, seq_len, d_model)

    Example:
        >>> import torch
        >>> model = MultiHeadAttention(d_model=512, context_size=32, num_heads=8)
        >>> input_tensor = torch.randn(16, 100, 512)
        >>> output_tensor = model(input_tensor)
        >>> print(output_tensor.shape)
        torch.Size([16, 100, 512])
    """

    def __init__(
        self,
        d_model: int,
        context_size: int,
        num_heads: int,
        dropout_pct: float = 0.1,
        qkv_bias: bool = False,
    ):
        super().__init__()

        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads  # Dim of each head
        self.context_size = context_size
        self.dropout = nn.Dropout(dropout_pct)
        self.register_buffer(
            "mask", torch.triu(torch.ones(context_size, context_size), diagonal=1)
        )

        self.query_W = nn.Linear(d_model, d_model, bias=qkv_bias)
        self.key_W = nn.Linear(d_model, d_model, bias=qkv_bias)
        self.value_W = nn.Linear(d_model, d_model, bias=qkv_bias)
        self.out_proj = nn.Linear(d_model, d_model)

    def _split_heads(self, x: Tensor) -> Tensor:
        """Split the features at each head by reshaping and transposing them.

        Returns:
            torch.Tensor: Tensor of shape (batch_size, num_heads, seq_length, head_dim).
        """
        # B, T, C
        batch_size, seq_len, _ = x.size()

        # After transposing: (B, n_heads, T, h_dim)
        x_split: Tensor = x.view(
            batch_size, seq_len, self.num_heads, self.head_dim
        ).transpose(1, 2)
        return x_split

    def _concat_heads(self, x: Tensor) -> Tensor:
        """
        Concatenates the heads of the input tensor along the last dimension.

        Args:
            x (torch.Tensor): Input tensor of shape (B, n_heads, T, h_dim).

        Returns:
            torch.Tensor: Concatenated tensor of shape (B, T, n_heads * h_dim).
        """
        B, n_heads, T, h_dim = x.size()
        # After transposing: (B, T, n_heads * h_dim)
        # self.d_model = n_heads * h_dim
        x_concat: Tensor = x.transpose(1, 2).contiguous().view(B, T, (n_heads * h_dim))
        return x_concat

    def forward(self, x: Tensor) -> Tensor:
        B, T, C = x.size()
        # Compute the query, key and value features
        # (B, T, C) @ (C, C) -> (B, T, C)
        queries: Tensor = self.query_W(x)  # (B, T, C)
        keys: Tensor = self.key_W(x)  # (B, T, C)
        values: Tensor = self.value_W(x)  # (B, T, C)

        # Split the features
        # C = n_heads * h_dim
        # (B, T, C) -> (B, n_heads, T, h_dim)
        queries = self._split_heads(queries)
        keys = self._split_heads(keys)
        values = self._split_heads(values)

        # Calculate the attention
        # (B, n_heads, T, h_dim) @ (B, n_heads, h_dim, T) -> (B, n_heads, T, T)
        attn_scores: Tensor = queries @ keys.transpose(-1, -2)  # (B, n_heads, T, T)
        # Mask the attention
        mask = self.mask.bool()[:T, :T]  # (T, T)
        attn_scores.masked_fill_(mask, float("-inf"))  # (B, n_heads, T, T)
        attn_weights: Tensor = F.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)  # (B, n_heads, T, T)

        # (B, n_heads, T, T) @ (B, n_heads, T, h_dim) -> (B, n_heads, T, h_dim)
        context_vectors: Tensor = attn_weights @ values  # (B, n_heads, T, h_dim)
        # Concatenate the attention and the features
        context_vectors = self._concat_heads(context_vectors)  # (B, T, n_heads * h_dim)
        # (B, T, C) @ (C, C) -> (B, T, C)
        context_vectors = self.out_proj(context_vectors)  # (B, T, C)
        return context_vectors

In [12]:
# (B, T, D) @ (D, D) -> (B, T, D)
torch.manual_seed(seed)

multi_head_attn: MultiHeadAttention = MultiHeadAttention(
    d_model=embedding_dim,
    context_size=context_size,
    num_heads=2,
    dropout_pct=0.1,
)
print(f"{input_seq.shape = }")
print(f"{multi_head_attn = }")
output: Tensor = multi_head_attn(input_seq)
print(f"{output.shape = }")

output

input_seq.shape = torch.Size([2, 8, 16])
multi_head_attn = MultiHeadAttention(
  (dropout): Dropout(p=0.1, inplace=False)
  (query_W): Linear(in_features=16, out_features=16, bias=False)
  (key_W): Linear(in_features=16, out_features=16, bias=False)
  (value_W): Linear(in_features=16, out_features=16, bias=False)
  (out_proj): Linear(in_features=16, out_features=16, bias=True)
)
output.shape = torch.Size([2, 8, 16])


tensor([[[-0.3474, -0.3107, -0.1027, -0.1821, -0.1602,  0.2133, -0.0108,
           0.0943,  0.2866,  0.0771,  0.0632, -0.0559, -0.0075,  0.0735,
          -0.1950, -0.4179],
         [-0.2610, -0.5279, -0.2566, -0.2172, -0.1152,  0.1345,  0.1930,
           0.1484,  0.1608, -0.0315, -0.0932, -0.1033, -0.1208,  0.0333,
          -0.2987, -0.4774],
         [-0.3143, -0.3279, -0.1460, -0.1762, -0.0125,  0.1674,  0.0904,
           0.1398,  0.2238, -0.0031,  0.0670, -0.1306, -0.0120,  0.1309,
          -0.2423, -0.3489],
         [-0.3131, -0.4945, -0.2172, -0.1274, -0.0659,  0.1363,  0.2305,
           0.2004,  0.2069, -0.0503, -0.0257, -0.1802, -0.0908,  0.0178,
          -0.3092, -0.4385],
         [-0.3075, -0.4451, -0.1925, -0.1408, -0.0238,  0.1458,  0.2024,
           0.1857,  0.1865, -0.0444,  0.0104, -0.1878, -0.0589,  0.0564,
          -0.2893, -0.3895],
         [-0.2897, -0.4864, -0.2115, -0.1478,  0.0254,  0.1132,  0.2349,
           0.1885,  0.1554, -0.0157, -0.0323, -0.201

# Implementing A GPT-model From Scratch

- Coding a GPT-like LLM that can be trained to generate uman-like text.
- Normalizing layer activations to stabilize NN training.
- Adding `shortcut connections` to train deep NNs effectively.
- Implementing transformer blocks to create GPT models of various sizes.
- Computing the number of parameters and storage requiremnts of GPT models.

In [20]:
tok_emb: nn.Embedding = nn.Embedding(embedding_dim, embedding_dim)

tok_emb(torch.tensor([[1, 2, 3]]))

tensor([[[-0.3479, -0.9514, -0.2711,  0.4511,  0.1239,  0.2274,  0.7453,
           1.2946,  0.6269, -0.2278,  1.2327, -0.6556,  0.7287,  0.8472,
          -1.6667,  1.0499],
         [ 0.0575,  1.2494,  0.5849,  0.5618,  0.1645,  0.5396, -1.3888,
           0.5524,  0.8906, -0.9869,  0.7470,  0.6496, -2.0915,  0.8895,
          -2.3194,  0.7862],
         [ 2.8540,  1.3330, -0.1396,  0.0471,  0.9951,  1.1297,  0.1833,
          -0.0111, -0.0110, -0.1556, -2.2002,  0.1646, -1.0052,  0.0985,
          -1.6470, -0.9355]]], grad_fn=<EmbeddingBackward0>)

In [21]:
torch.tensor([[1, 2, 3]]).shape

torch.Size([1, 3])